In [25]:
import pandas as pd

gen_recipes_df = pd.read_json('data/879-recipes-output-v2.jsonl', lines=True)[['gens_article', 'title']]
raw_recipes_df = pd.read_csv('data/879-recipes-sam-sifton.csv', index_col=0)

In [26]:
gen_recipes_df.head()

,gens_article,title
0,[Photo\nDespite the delicate sensuality of the...,Beef Suya
1,[Jollof rice may be one of the simplest and be...,Jollof Rice
2,[It’s spring and everyone wants to jump on the...,Puff Puff
3,[The franchise shares with us the fruits of it...,Agege Bread
4,[What You’ll Need\n• Large container of tofu o...,Crispy Yam Fries


In [27]:
raw_recipes_df.head()

,Name,Author,Image URL,Cooking Time
0,Fettuccine With Ricotta and a Fistful of Mint,Sam Sifton,https://static01.nyt.com/images/2019/02/17/din...,NaN
1,Pasta With Sausage and Parm,Sam Sifton,https://static01.nyt.com/images/2019/02/17/din...,NaN
2,New Mexican Hot Dish,Sam Sifton,https://static01.nyt.com/images/2019/02/17/din...,NaN
3,Speedy Fish Chowder,Sam Sifton,https://static01.nyt.com/images/2019/02/17/din...,NaN
4,Weeknight Fried Rice,Sam Sifton,https://static01.nyt.com/images/2019/02/17/din...,NaN


In [28]:
merged_df = raw_recipes_df.merge(gen_recipes_df, left_on='Name', right_on='title').drop(columns=['title'])
merged_df = merged_df.rename(columns={"gens_article": "Generated Article"})
merged_df['Generated Article'] = [x[0] for x in merged_df['Generated Article']]
merged_df.head()

,Name,Author,Image URL,Cooking Time,Generated Article
0,Beef Suya,Yewande Komolafe,https://static01.nyt.com/images/2019/06/26/din...,"30 minutes, plus marinating",Photo\nDespite the delicate sensuality of the ...
1,Jollof Rice,Yewande Komolafe,https://static01.nyt.com/images/2019/06/26/din...,1 1/2 hours,Jollof rice may be one of the simplest and bes...
2,Puff Puff,Yewande Komolafe,https://static01.nyt.com/images/2019/06/26/din...,"30 minutes, plus rising",It’s spring and everyone wants to jump on the ...
3,Agege Bread,Yewande Komolafe,https://static01.nyt.com/images/2019/06/26/din...,"1 1/2 hours, plus rising and cooling",The franchise shares with us the fruits of its...
4,Crispy Yam Fries,Yewande Komolafe,https://static01.nyt.com/images/2019/06/26/din...,40 minutes,What You’ll Need\n• Large container of tofu or...


In [29]:
merged_df.iloc[3]['Generated Article']

'The franchise shares with us the fruits of its labour\nAny mealworm can be referred to as an agege bread. An agege bread is a simple and delicious barley rice and chickpea pattie with four kinds of grains: barley, wheat, or ramush are examples.\nThe bread has all the qualities of a family meal: it is easy to work with and easy to prepare. It is nutritious, filling, and delicious.\nAnd the best thing is that, it only has to be made once! Once is all it takes to start making these great healthy patties. For the record, each agege bread has a handcrafted message for you, and most are subject to a three-month change.\nThis bowl looks majestic but still packs an intimidating nutritional punch.\nPreparation time: 5-10 minutes\nRoughly 20 minutes cooking time\nHands on time: 1 minute\nIngredients\n200 grams wide wide noodles, cooked\nFor the bread\n50 grams broken wheat flour\n50 grams graham cracker crumbs\n250 grams cooked agesge grain\n1 teaspoon salt\n1 teaspoon tapioca flour\n1 tablespo

In [30]:
import string

def parse_to_markdown(article_str, fname, title, date, author, description, imageURL):
#     article_str = article_str.replace("\nIngredients", "\n### Ingredients")
#     article_str = article_str.replace("\nDirections", "\n### Directions")
    
    ingredient_idx = article_str.find('\nIngredients')
    direction_idx = article_str.find('\nDirections')
    ingredient_len = len('\nIngredients')
    direction_len = len('\nDirections')
    
    pre_ingredient_prattle = article_str[:ingredient_idx]
    ingredients_listed = article_str[ingredient_idx+ingredient_len:direction_idx]
    directions_listed = article_str[direction_idx+direction_len:]
    
    new_prattle = "*" + '*\n\n*'.join(pre_ingredient_prattle.split('\n')) + "*"
    new_ingredients = "" 
    for ing in ingredients_listed.split('\n'):
        if len(ing) > 0:
            if ing[0] in string.digits:
                new_ingredients += "\n\n* " + ing
            else:
                new_ingredients += "\n\n" + ing
                           
    new_directions = '\n\n1. '.join(directions_listed.split('\n'))

    metadata = """---
title: {}
date: "2019-08-01T23:46:37.121Z"
template: "post"
draft: false
slug: "/posts/{}/"
category: "Crazy Recipes"
tags:
- "AI"
- "{}"
description: "A tasty AI-generated recipe. Endorsed by Obama."
socialImage: "{}"
---

![]({})

""".format(title, '-'.join(title.split(' ')), author, imageURL, imageURL)
    
    new_str = metadata + new_prattle + "\n### Ingredients" + new_ingredients + "\n### Directions" + new_directions
    
    with open(fname, 'w') as fobject:
        fobject.write(new_str)
        fobject.close()
    
def generate_all_recipes(idx_list):
    for idx in idx_list:
        row = merged_df.iloc[idx]
        article_str = row['Generated Article']
        fpath = "../content/posts/"
        fname = fpath + '-'.join(row['Name'].split(' ')) + '.md'
        try:
            parse_to_markdown(article_str, fname, title=row['Name'], date=None, 
                              author=row['Author'], description=None, imageURL=row['Image URL'])
        except:
            print("Recipe couldn't be parsed.")
    
good_idx = merged_df['Generated Article'].str.contains('\nIngredients') & merged_df['Generated Article'].str.contains('\nDirections')
idx_list = [i for i,x in enumerate(good_idx) if x]
generate_all_recipes(idx_list)


Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.
Recipe couldn't be parsed.


In [31]:
merged_df['Generated Article'].str.contains('\nIngredients').sum()

175

In [32]:
merged_df['Generated Article'].str.contains('\nDirections').sum()

139

In [33]:
good_idx = merged_df['Generated Article'].str.contains('\nIngredients') & merged_df['Generated Article'].str.contains('\nDirections')
sum(good_idx)

121